## Calculating VPIN CDF for Banking Stocks: JPM, BAC, C, USB, WFC

### Importing necessary packages

In [ ]:
import itertools
import numpy as np
import datetime
import pandas as pd   
from IPython.core.debugger import Tracer
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score
import gc
from sklearn.metrics import classification_report
from scipy.stats.stats import pearsonr

### Function to aggregate the trading volume into volume buckets and derive the Buy and Sell volume based on return 

In [ ]:
def get_buckets(df,bucketSize):
    volumeBuckets = pd.DataFrame(columns=['Buy','Sell','Time'])
    count = 0
    BV = 0
    SV = 0
    for index,row in df.iterrows():
        newVolume = row['volume']
        z = row['z']
        if bucketSize < count + newVolume:
            BV = BV + (bucketSize-count)*z
            SV = SV + (bucketSize-count)*(1-z)
            volumeBuckets = volumeBuckets.append({'Buy':BV, 'Sell':SV, 'Time':index},ignore_index=True)
            count = newVolume-(bucketSize-count)
            if int(count/bucketSize) > 0:
                for i in range(0,int(count/bucketSize)):
                    BV = (bucketSize)*z
                    SV = (bucketSize)*(1-z)
                    volumeBuckets = volumeBuckets.append({'Buy':BV, 'Sell':SV, 'Time':index},ignore_index=True)

            count = count%bucketSize
            BV = (count)*z
            SV = (count)*(1-z)
        else:
            BV = BV + (newVolume)*z
            SV = SV + (newVolume)*(1-z)
            count = count + newVolume

    volumeBuckets = volumeBuckets.set_index('Time')
    return volumeBuckets

### Function to calculate the VPIN and CDF metrics from Buy and Sell Volumes

In [ ]:
def calc_vpin(data, bucketSize,window):
    
    volume = (data['SIZE'])
    trades = (data['PRICE'])
    
    trades_1min = trades.diff(1).resample('1min', how='sum').dropna()
    volume_1min = volume.resample('1min', how='sum').dropna()
    sigma = trades_1min.std()
    z = trades_1min.apply(lambda x: norm.cdf(x/sigma))
    df = pd.DataFrame({'z': z, 'volume': volume_1min}).dropna()
    
    volumeBuckets=get_buckets(df,bucketSize)
    volumeBuckets['VPIN'] = abs(volumeBuckets['Buy']-volumeBuckets['Sell']).rolling(window).mean()/bucketSize
    volumeBuckets['CDF'] = volumeBuckets['VPIN'].rank(pct=True)
    
    return volumeBuckets

### Reading trade date of the specific stocks

In [ ]:
sym = ['C','BAC','USB','JPM','WFC']
df={}; sec_trades = {}
for s in sym:
    f = s + '.csv'
    print('Reading '+f)
    sec_trades[s] = pd.read_csv(f,parse_dates=[['DATE','TIME_M']],index_col=[0])

### Setting bucket size and calculating VPINs and CDFs for each stock

In [ ]:
volume = {'C':300000,'USB':100000,'JPM':200000,'BAC':1250000,'WFC':300000}
for s in sym:
    print('Calculating VPIN')
    df[s] = calc_vpin(sec_trades[s],volume[s],50)
    print(s+' '+str(df[s].shape))
    
avg = pd.DataFrame()
print(avg.shape)
metric = 'CDF'
avg[metric] = np.nan
for stock,frame in df.items():
    frame = frame[[metric]].reset_index().drop_duplicates(subset='Time', keep='last').set_index('Time')
    avg = avg.merge(frame[[metric]],left_index=True,right_index=True,how='outer',suffixes=('',stock))
    print(avg.shape)
avg = avg.dropna(axis=0,how='all').fillna(method='ffill')

avg.to_csv('CDF.csv')

### Calculating the average rolling correlation between VPIN CDF of banking stocks 

In [ ]:
fields = ['Time','CDFC','CDFBAC','CDFUSB','CDFJPM','CDFWFC']
df = pd.read_csv('CDF.csv',parse_dates=['Time'],index_col=[0],usecols = fields)

rolling_pariwise_corr = pd.rolling_corr(df,window=50,pairwise=True)

thres = pd.DataFrame()
thres['AvgCorrAssets'] = rolling_pariwise_corr.groupby(by=['Time']).sum().sum(axis=1)/((len(fields)-1)**2)
thres.to_csv('AvgCorrAssets.csv')

## Calculating VPIN CDF for BAC on individual exchanges and deriving the average rolling correlations between them

### Calculating VPIN for each exchange based on different buckets taking average daily volume

In [ ]:
sec_trades = pandas.read_csv('BAC.csv',parse_dates=[['DATE','TIME_M']],index_col=[0])
print('File read complete')

exchanges = sec_trades['EX'].unique()
exchanges = exchanges[:-2]
bucketsize_standard = 100000
rolling_window = 50
df_list = list()
df_vpin_list = list()
volume_exchanges = list()
bucketsize = list()
for i in range(len(exchanges)-1):
    df_list.append(sec_trades[sec_trades['EX'] == exchanges[i]])
for i in range(len(exchanges)):
    volume_exchanges.append(df_list[i]['SIZE'].sum())
nbuckets = 6574
for i in range(len(exchanges)):
    bucketsize = int(volume_exchanges[i]/nbuckets)
    df_vpin_list.append(calc_vpin(df_list[i],bucketsize,rolling_window))
    
    
avg = pd.DataFrame()
metric = 'VPIN'
avg[metric] = np.nan
for i in range(len(exchanges)):
    print(exchanges[i])
    frame = df_vpin_list[i]
    frame = frame[[metric]].reset_index().drop_duplicates(subset='Time', keep='last').set_index('Time')
    avg = avg.merge(frame[[metric]],left_index=True,right_index=True,how='outer',suffixes=('',exchanges[i]))
    print(avg.shape)
avg = avg.dropna(axis=0,how='all').fillna(method='ffill')
del avg['VPIN']
avg = avg.dropna()
print(avg)

rolling_pariwise_corr = pd.rolling_corr(avg,window=50,pairwise=True)
thres = pd.DataFrame()
thres['AvgCorrEx'] = rolling_pariwise_corr.groupby(by=['Time']).sum().sum(axis=1)/(len(exchanges)**2)
print(thres.tail())
thres.to_csv('AvgCorrEx.csv')

## Calculating Quote Imbalance based on Quotes data for BAC

### read and clean BAC quotes

In [ ]:
def cleanup(x):
    x=x[x['SYM_SUFFIX'].isnull()]
    return x

In [ ]:
sec_quotes = pandas.read_csv('BAC_QF.csv',parse_dates=[['DATE','TIME_M']],index_col=[0])
sec_quotes=cleanup(sec_quotes)

### Quote Imbalance function which calculates the average difference in volume of bid quotes and ask quotes among all exchanges

In [ ]:
def imbalance(sec_quotes):
    bids={}
    bids_vol={}
    asks={}
    asks_vol={}
    sec_bids=sec_quotes[sec_quotes['BID']>0]
    sec_bids=sec_bids[sec_bids['BIDSIZ']>0]
    sec_asks=sec_quotes[sec_quotes['ASK']>0]
    sec_asks=sec_asks[sec_quotes['ASKSIZ']>0]
    for ex in sec_quotes['EX'].unique():
        bids[ex]=(sec_bids[sec_bids['EX']==ex]['BID'])
        bids_vol[ex]=(sec_bids[sec_bids['EX']==ex]['BIDSIZ'])

        asks[ex]=(sec_asks[sec_asks['EX']==ex]['ASK'])
        asks_vol[ex]=(sec_asks[sec_asks['EX']==ex]['ASKSIZ'])

    
    df_comb=pd.DataFrame()
    for ex in sec_quotes['EX'].unique():
        df=pd.DataFrame()
        df1=pd.DataFrame()
        bidquote_1min = bids[ex].resample('1min', how='last').ffill().fillna(0)
        bidvol_1min = bids_vol[ex].resample('1min', how='last').ffill().fillna(0)
        askquote_1min = asks[ex].resample('1min', how='last').ffill().fillna(0)
        askvol_1min = asks_vol[ex].resample('1min', how='last').ffill().fillna(0)
        df1=pd.concat([bidquote_1min, bidvol_1min,askquote_1min,askvol_1min], join='outer', axis=1)
        df1=df1.ffill().fillna(0)
        df['bprice_'+ex]=df1['BID']
        df['bvol_'+ex]=df1['BIDSIZ']
        df['aprice_'+ex]=df1['ASK']
        df['avol_'+ex]=df1['ASKSIZ']

        if df_comb.empty:
            df_comb=df.copy()
            df_comb['avg_bid']=df['bprice_'+ex]*df['bvol_'+ex]
            df_comb['price_bid']=df['bprice_'+ex]
            df_comb['avg_ask']=df['aprice_'+ex]*df['avol_'+ex]
            df_comb['price_ask']=df['aprice_'+ex]
            n1=df_comb['bprice_'+ex].apply(lambda x: int(x!=0))
            n2=df_comb['aprice_'+ex].apply(lambda x: int(x!=0))
        else:
            df_comb=df_comb.merge(df, how='outer', right_index=True, left_index=True).ffill().fillna(0)
            df_comb['avg_bid']=df_comb['avg_bid']+df_comb['bprice_'+ex]*df_comb['bvol_'+ex]
            df_comb['price_bid']=df_comb['price_bid']+df_comb['bprice_'+ex]
            df_comb['avg_ask']=df_comb['avg_ask']+df_comb['aprice_'+ex]*df_comb['avol_'+ex]
            df_comb['price_ask']=df_comb['price_ask']+df_comb['aprice_'+ex]
            n1=n1+df_comb['bprice_'+ex].apply(lambda x: int(x!=0))
            n2=n2+df_comb['aprice_'+ex].apply(lambda x: int(x!=0))

    df_comb['avg_bid']=df_comb['avg_bid']*n1/df_comb['price_bid']
    df_comb['avg_ask']=df_comb['avg_ask']*n2/df_comb['price_ask']
    imbalance=df_comb['avg_bid']-df_comb['avg_ask']
    return imbalance

quote_imb=imbalance(sec_quotes)
quote_imb.to_csv('BACquote_imb.csv')

## Applying Theory Based and ML Based Trading Strategies

### Merging and processing data for Trading Strategy

In [ ]:
df_corr_assets = pd.read_csv('req_files/AvgCorrAssets.csv',parse_dates=['Time'],index_col='Time')
df_corr_ex = pd.read_csv('req_files/AvgCorrEx.csv',parse_dates=['Time'],index_col='Time')
df_vpin = pd.read_csv('req_files/BACVPIN.csv',parse_dates=['Time'],usecols=['Time','CDF'],index_col='Time')
df_quote_imb = pd.read_csv('req_files/imbalance.csv',parse_dates=['TIME'],index_col='TIME')
df_price = pd.read_csv('req_files/BACprice.csv',parse_dates=['DATE_TIME_M'],index_col='DATE_TIME_M')

In [ ]:
total_df = pd.DataFrame()
frame = df_corr_assets[[df_corr_assets.columns[0]]].reset_index().drop_duplicates(subset='Time', keep='last').set_index('Time')
total_df = total_df.merge(frame[[frame.columns[0]]],left_index=True,right_index=True,how='outer')
print (total_df.shape)

frame = df_corr_ex[[df_corr_ex.columns[0]]].reset_index().drop_duplicates(subset='Time', keep='last').set_index('Time')
total_df = total_df.merge(frame[[frame.columns[0]]],left_index=True,right_index=True,how='outer')
print (total_df.shape)

frame = df_vpin[[df_vpin.columns[0]]].reset_index().drop_duplicates(subset='Time', keep='last').set_index('Time')
total_df = total_df.merge(frame[[frame.columns[0]]],left_index=True,right_index=True,how='outer')
print (total_df.shape)

frame = df_quote_imb[[df_quote_imb.columns[0]]].reset_index().drop_duplicates(subset='TIME', keep='last').set_index('TIME')
total_df = total_df.merge(frame[[frame.columns[0]]],left_index=True,right_index=True,how='outer')
print (total_df.shape)

frame = df_price[[df_price.columns[0]]].reset_index().drop_duplicates(subset='DATE_TIME_M', keep='last').set_index('DATE_TIME_M')
total_df = total_df.merge(frame[[frame.columns[0]]],left_index=True,right_index=True,how='outer')
print (total_df.shape)

total_df = total_df.dropna(axis=0,how='all').fillna(method='ffill')
total_df = total_df.dropna(how='any')
print (total_df.shape)

In [ ]:
total_df.to_csv('data_qlearner.csv')

### Processing merged data

In [ ]:
df= pd.read_csv('data_qlearner_raw.csv', parse_dates=['Time'],index_col='Time' )
df=df[df['zcorras'].notnull()]

In [ ]:
df['returns']=df['PRICE'].pct_change()
df['returns']=df['returns'].shift(-1)
df=df.replace([np.inf, -np.inf], np.nan)
df=df.ffill()
df = df[:-1]
df=df.between_time('8:00','16:00')
df=df[df.index.dayofweek < 5]
df=df[df.index<pd.to_datetime('2016-12-30')]
df.to_csv('new_data.csv')

savg=df[['AvgCorrEx','AvgCorrAssets','quote_imb']].rolling(window=100).mean()
sstd=df[['AvgCorrEx','AvgCorrAssets','quote_imb']].rolling(window=100).std()

### Calculating z-scores for input features using rolling mean and standard deviation; and windsoring z-scores

In [ ]:
zcorrex=(df['AvgCorrEx']-savg['AvgCorrEx'])/sstd['AvgCorrEx']
zcorras=(df['AvgCorrAssets']-savg['AvgCorrAssets'])/sstd['AvgCorrAssets']
zimb=(df['quote_imb']-savg['quote_imb'])/sstd['quote_imb']

zcorrex[zcorrex>3]=3
zcorrex[zcorrex<-3]=-3
zcorras[zcorras>3]=3
zcorras[zcorras<-3]=-3
zimb[zimb>3]=3
zimb[zimb<-3]=-3


df_trading=pd.DataFrame({'price':df['PRICE'][100:],'cdf':df['CDF'][100:],'zcorrex':zcorrex[100:],'zcorras':zcorras[100:],'zimb':zimb[100:],'returns':df['returns'][100:], 'Time':df.index[100:]}
                        ).set_index('Time')
df_trading.to_csv('final.csv')

### Applying the theory based strategy

In [ ]:
pos=pd.DataFrame(columns=['position','returns','Time'])
position=0

for index,row in df_trading.iterrows():
    if(row['cdf']>0.5 and row['zcorrex']+row['zcorras']>4):
        if(row['zimb']>1.5):
            if(position<0):
                position=0
            position+=0.1
            bcount=10
        if(row['zimb']<-1.5):
            if(position>0):
                position=0
            position-=0.1
            scount=10
    if(position>0):
        bcount=bcount-1
        if(bcount==0):
            position=0
    elif(position<0):
        scount=scount-1
        if(scount==0):
            position=0
    pos=pos.append({'position':position,'returns':row['returns'], 'price':row['price'],'Time':index},ignore_index=True)
    #print('position :'+ str(position))
pos=pos.set_index('Time')
pos['wealth']=(1+pos['returns']*pos['position']).cumprod()

In [ ]:
ax=pos[['position','wealth']].plot(secondary_y=['wealth'])
ax.figure

### Applying ML Classification Based Strategies

In [ ]:
df= pd.read_csv('final.csv', parse_dates=['Time'],index_col='Time' )
df=df[df['zcorras'].notnull()]

### Discretizing returns data and creating Training/ Test split 

In [ ]:
X = df[['cdf','zcorras','zcorrex','zimb']]
y=(df['returns']>0).astype(int)

# Split the data into training/testing sets
df_X_train = X[:10000]
df_X_test = X[10000:]

# Split the targets into training/testing sets
df_y_train = y[:10000]
df_y_test = y[10000:]

### Fitting the data to different classifiers and obtaining the detailed stats for each

In [ ]:
score=[]
pos=None
classifiers = [
    KNeighborsClassifier(4),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=0.25, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=4),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

for cla,name in zip(classifiers,names): 
    clf=cla
    clf.fit(df_X_train, df_y_train)
    score.append(accuracy_score(df_y_test,clf.predict(df_X_test)))
    position=((clf.predict_proba(df_X_test))*[-0.05,0.05]).sum(axis=1)
    print(classification_report(df_y_test, clf.predict(df_X_test)))
    wealth=(1+df['returns'][10000:]*position).cumprod()
    if(pos is None):
        pos=pd.DataFrame({'returns':df['returns'][10000:], name:wealth,'price':df['price'][10000:],'Time':df.index[10000:]}).set_index('Time')
    else:
        pos[name]=position
    ax=pos[[name,'price']].plot(secondary_y=[name])
    ax.figure
    gc.collect()